In [ ]:
%matplotlib inline
import os, sys
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
import ccdproc as cp
from ccdproc import CCDData
import numpy as np
import glob, subprocess
import datetime


In [ ]:
def create_directory(path):
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        print('Directory {:s} already exists.'.format(path))
    return None

def compress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/fpack',filename])
    os.remove(filename)
    return None

def decompress_file_fpack(filename):
    process = subprocess.call(['/Users/hhsieh/Astro/tools/cfitsio/funpack',filename])
    os.remove(filename)
    return None

def compress_directory_fpack(file_path):
    print('>>> Starting fpack compression of FITS files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fits_file in glob.glob('*.fits'):
        compress_file_fpack(fits_file)
    print('>>> fpack compression of FITS files complete.')
    return None
    
def decompress_directory_fpack(file_path):
    print('>>> Starting decompression of fz files in {:s}...'.format(file_path))
    os.chdir(file_path)
    for fz_file in glob.glob('*.fz'):
        decompress_file_fpack(fz_file)
    print('>>> Decompression of fz files complete.')
    return None


In [ ]:
def get_date_id():
    # obtain YYYYMMDD date from filename of GMOS FITS files being processed
    for raw_mef_file in glob.glob('*.fits'):
        fits_date = raw_mef_file[1:9]
    return fits_date

def flatfield_median_combine(flatfield_name,reduction_log):
    print('{:s} - Starting median combination of flatfield frames...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Starting median combination of flatfield frames...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip1.fits'
    for fits_file in glob.glob('*.chip1.otz.fits'):
        with fits.open(fits_file) as hdulist:
            fits_hdr,fits_data = hdulist[0].header,hdulist[0].data
            fits_mean   = np.mean(fits_data)
            fits_data_n = fits_data / fits_mean
            #file_prefix = fits_file[0:19]
            #fits_file_n = file_prefix + '_n.otz.fits'
            fits_file_n = fits_file[0:19] + '_n.otz.fits'
            fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        os.remove(fits_file_n)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 1 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 1 complete.\n')

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip2.fits'
    for fits_file in glob.glob('*.chip2.otz.fits'):
        with fits.open(fits_file) as hdulist:
            fits_hdr,fits_data = hdulist[0].header,hdulist[0].data
            fits_mean   = np.mean(fits_data[251:1012,1:2042])
            fits_data_n = fits_data / fits_mean
            fits_file_n = fits_file[0:19] + '_n.otz.fits'
            fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        os.remove(fits_file_n)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 2 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 2 complete.\n')

    flatfield_list = []
    output_flat_filename = flatfield_name + '.chip3.fits'
    for fits_file in glob.glob('*.chip3.otz.fits'):
        with fits.open(fits_file) as hdulist:
            fits_hdr,fits_data = hdulist[0].header,hdulist[0].data
            fits_mean   = np.mean(fits_data)
            fits_data_n = fits_data / fits_mean
            fits_file_n = fits_file[0:19] + '_n.otz.fits'
            fits.writeto(fits_file_n,fits_data_n,fits_hdr)
        fits_data = CCDData.read(fits_file_n)
        flatfield_list.append(fits_data)
        os.remove(fits_file_n)
    master_flat = cp.combine(flatfield_list,method='median')
    master_flat.write(output_flat_filename)
    print('>>> Median combination of flatfield frames for Chip 3 complete.')
    reduction_log.write('>>> Median combination of flatfield frames for Chip 3 complete.\n')

    print('{:s} - Median combination of flatfield frames complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))
    reduction_log.write('{:s} - Median combination of flatfield frames complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')))

    return None


In [ ]:
def process_flatfield_directories(basewd):
    filter_id = basewd[-2:-1]
    instrname = 'not recognized'
    if basewd[-19:-18] == 'N': instrname = 'gmosn'
    if basewd[-19:-18] == 'S': instrname = 'gmoss'

    rawfits_dir  = '/rawfits_twiskyflat/'
    rawbias_dir  = '/rawfits_bias/'
    procfits_dir = '/procfits_twiskyflat/'
    exclfits_dir = '/procfits_twiskyflat/exclude/'

    os.chdir(basewd)
    for datadir in sorted(glob.glob('ut*_gemini?_twiskyflat')):
        path_rawfits  = basewd + datadir + rawfits_dir
        path_rawbias  = basewd + datadir + rawbias_dir
        path_procfits = basewd + datadir + procfits_dir
        path_exclfits = basewd + datadir + exclfits_dir
    
        flatfield_name = 'n' + datadir[2:10] + '.' + instrname + '.twiskyflat.' + filter_id
        flatfits_file_chip2 = path_procfits + flatfield_name + '.chip2.fits.fz'
        reduction_logfile = basewd + datadir + '/log_reduction_' + datadir[2:10] + '_' + instrname + '.txt'
    
        if not os.path.exists(flatfits_file_chip2):
            if os.path.exists(path_procfits):
                if os.path.exists(path_exclfits):
                    if instrname != 'not recognized':
                        with open(reduction_logfile,'w') as reduction_log:
                            print('\n{:s} - Starting median combination of flatfield files in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                            reduction_log.write('\n{:s} - Starting median combination of flatfield files in {:s}...\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                            os.chdir(path_procfits)
                            decompress_directory_fpack(path_procfits)                
                            flatfield_median_combine(flatfield_name,reduction_log)
                            compress_directory_fpack(path_procfits)     
                            print('{:s} - Processing flatfield files in {:s} complete.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                            reduction_log.write('\n{:s} - Processing flatfield files in {:s} complete.\n'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                    else:
                        print('{:s} - Instrument for {:s} not recognized.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
                else:
                    print('{:s} - Flatfield files in {:s} not yet screened.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
            else:
                print('{:s} - Flatfield files in {:s} not yet processed.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))
        else:
            print('{:s} - Median flatfields for {:s} already generated.'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),datadir))

    return None


In [ ]:
process_flatfield_directories(base_path)
